# Keyboard Hints

> Components for displaying keyboard shortcut hints to users.

In [ ]:
#| default_exp components.hints

In [ ]:
#| export
from __future__ import annotations
from collections import defaultdict
from typing import Union
from fasthtml.common import Div, Span, FT

from cjm_fasthtml_keyboard_navigation.core.actions import KeyAction
from cjm_fasthtml_keyboard_navigation.core.manager import ZoneManager

from cjm_fasthtml_daisyui.components.data_display.badge import badge, badge_colors, badge_styles
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import (
    flex_display, flex_wrap, gap, items
)
from cjm_fasthtml_tailwind.utilities.typography import font_size, font_family, font_weight
from cjm_fasthtml_tailwind.utilities.spacing import m
from cjm_fasthtml_tailwind.core.base import combine_classes

from cjm_fasthtml_lucide_icons.factory import lucide_icon

## Single Hint Badge

In [ ]:
#| export
# Icon mappings for navigation patterns and common keys
NAV_ICON_MAP = {
    "up_down": "arrow-down-up",
    "left_right": "arrow-left-right",
}

KEY_ICON_MAP = {
    "shift": "arrow-big-up",
    "delete": "trash-2",
    "del": "trash-2",
    "enter": "corner-down-left",
    "return": "corner-down-left",
    "backspace": "delete",
    "bksp": "delete",
    "escape": "x",
    "esc": "x",
}


def get_key_icon(
    key_name: str,    # key name to look up (case-insensitive)
    size: int = 3     # icon size
) -> FT | None:       # icon component or None if no icon mapping
    """Get a lucide icon for a key name, if one exists."""
    icon_name = KEY_ICON_MAP.get(key_name.lower())
    if icon_name:
        return lucide_icon(icon_name, size=size)
    return None


def render_hint_badge(
    key_display: Union[str, FT],  # formatted key string or icon component
    description: str,              # action description
    style: str = "ghost",          # badge style (ghost, outline, soft, dash)
    auto_icon: bool = False        # auto-convert known keys to icons
) -> Div:                          # hint badge component
    """Render a single keyboard hint as a badge."""
    badge_style = getattr(badge_styles, style, badge_styles.ghost)
    
    # Handle key display
    if isinstance(key_display, str):
        # Try to auto-convert to icon if enabled
        if auto_icon:
            icon = get_key_icon(key_display)
            if icon:
                key_component = icon
            else:
                key_component = Span(key_display, cls=combine_classes(font_family.mono, font_weight.bold))
        else:
            key_component = Span(key_display, cls=combine_classes(font_family.mono, font_weight.bold))
    else:
        key_component = key_display
    
    return Div(
        key_component,
        Span(description, cls=m.l(1)),
        cls=combine_classes(badge, badge_style, flex_display, items.center, gap(1))
    )


def create_nav_icon_hint(
    icon_name: str,      # lucide icon name (e.g., "arrow-down-up")
    description: str,    # action description
    style: str = "ghost" # badge style
) -> Div:                # hint badge with icon
    """Create a hint badge with a lucide icon."""
    icon = lucide_icon(icon_name, size=3)
    return render_hint_badge(icon, description, style)


def create_modifier_key_hint(
    modifier: str,       # modifier key name (e.g., "shift", "ctrl")
    key_icon_or_text: Union[str, FT],  # the main key icon or text
    description: str,    # action description
    style: str = "ghost" # badge style
) -> Div:                # hint badge with modifier + key
    """Create a hint badge with a modifier key and main key."""
    badge_style = getattr(badge_styles, style, badge_styles.ghost)
    
    # Get modifier icon if available
    mod_icon = get_key_icon(modifier)
    mod_component = mod_icon if mod_icon else Span(modifier.capitalize(), cls=font_weight.bold)
    
    # Handle main key
    if isinstance(key_icon_or_text, str):
        main_icon = get_key_icon(key_icon_or_text)
        if main_icon:
            key_component = main_icon
        else:
            key_component = Span(key_icon_or_text, cls=combine_classes(font_family.mono, font_weight.bold))
    else:
        key_component = key_icon_or_text
    
    return Div(
        mod_component,
        key_component,
        Span(description, cls=m.l(1)),
        cls=combine_classes(badge, badge_style, flex_display, items.center, gap(1))
    )

In [ ]:
# Test hint badge with string
from fasthtml.common import to_xml

hint_badge = render_hint_badge("Space", "Select")
html = to_xml(hint_badge)
assert "Space" in html
assert "Select" in html
assert "badge" in html

# Test hint badge with icon
icon_hint = create_nav_icon_hint("arrow-down-up", "Navigate")
html = to_xml(icon_hint)
assert "Navigate" in html
assert "svg" in html  # Icon is an SVG

# Test auto_icon conversion
delete_hint = render_hint_badge("Delete", "Remove item", auto_icon=True)
html = to_xml(delete_hint)
assert "svg" in html  # Should convert to trash icon
assert "Remove item" in html

# Test get_key_icon
assert get_key_icon("shift") is not None
assert get_key_icon("delete") is not None
assert get_key_icon("unknown_key") is None

# Test modifier key hint (Shift + arrow)
shift_nav = create_modifier_key_hint("shift", lucide_icon("arrow-down-up", size=3), "Reorder")
html = to_xml(shift_nav)
assert "svg" in html
assert "Reorder" in html

## Hint Group

In [ ]:
#| export
def render_hint_group(
    group_name: str,         # group header text
    hints: list[tuple[str, str]],  # list of (key_display, description) tuples
    badge_style: str = "ghost"     # badge style for this group
) -> Div:                    # group container with header and hints
    """Render a group of related keyboard hints."""
    badges = [render_hint_badge(key, desc, badge_style) for key, desc in hints]
    
    return Div(
        Span(
            group_name,
            cls=combine_classes(font_size.xs, font_weight.semibold, m.r(2))
        ),
        *badges,
        cls=combine_classes(flex_display, items.center, gap(1), flex_wrap.wrap)
    )

In [ ]:
# Test hint group with string keys
group = render_hint_group(
    "Navigation",
    [("W/S", "Move"), ("A/D", "Switch")]
)
html = to_xml(group)
assert "Navigation" in html
assert "Move" in html
assert "Switch" in html

## Hints from Actions

In [ ]:
#| export
def group_actions_by_hint_group(
    actions: tuple[KeyAction, ...]  # actions to group
) -> dict[str, list[KeyAction]]:    # grouped actions
    """Group actions by their hint_group attribute."""
    groups = defaultdict(list)
    for action in actions:
        if action.show_in_hints and action.description:
            groups[action.hint_group].append(action)
    return dict(groups)

In [ ]:
#| export
def render_hints_from_actions(
    actions: tuple[KeyAction, ...],  # actions to display hints for
    badge_style: str = "ghost"       # badge style
) -> Div:                            # container with all hint groups
    """Render keyboard hints from action configurations."""
    groups = group_actions_by_hint_group(actions)
    
    group_components = []
    for group_name, group_actions in groups.items():
        hints = [(a.get_display_key(), a.description) for a in group_actions]
        group_components.append(
            render_hint_group(group_name, hints, badge_style)
        )
    
    return Div(
        *group_components,
        cls=combine_classes(flex_display, flex_wrap.wrap, gap(4))
    )

In [ ]:
# Test hints from actions
actions = (
    KeyAction(key=" ", htmx_trigger="x", description="Select", hint_group="Selection"),
    KeyAction(key="Delete", htmx_trigger="y", description="Remove", hint_group="Actions"),
    KeyAction(key="Enter", js_callback="z", description="Open", hint_group="Actions"),
    KeyAction(key="Backspace", htmx_trigger="w", description="Delete", show_in_hints=False),  # Hidden
)

hints_component = render_hints_from_actions(actions)
html = to_xml(hints_component)
assert "Selection" in html
assert "Actions" in html
assert "Select" in html
assert "Remove" in html
assert "Delete" not in html  # show_in_hints=False

## Full Keyboard Hints

In [ ]:
#| export
def render_keyboard_hints(
    manager: ZoneManager,                      # the zone manager
    include_navigation: bool = True,           # include navigation hints
    include_zone_switch: bool = True,          # include zone switching hints
    badge_style: str = "ghost",                # badge style
    container_id: str = "kb-hints",            # container element ID
    use_icons: bool = True                     # use lucide icons for nav hints
) -> Div:                                      # complete hints component
    """Render complete keyboard hints for a zone manager."""
    from cjm_fasthtml_keyboard_navigation.core.key_mapping import format_key_for_display
    
    sections = []
    
    # Navigation hints
    if include_navigation:
        if use_icons:
            nav_badge = create_nav_icon_hint(NAV_ICON_MAP["up_down"], "Navigate", badge_style)
            sections.append(
                Div(
                    Span("Navigation", cls=combine_classes(font_size.xs, font_weight.semibold, m.r(2))),
                    nav_badge,
                    cls=combine_classes(flex_display, items.center, gap(1), flex_wrap.wrap)
                )
            )
        else:
            nav_hints = [("↑/↓", "Navigate")]
            sections.append(render_hint_group("Navigation", nav_hints, badge_style))
    
    # Zone switching hints
    if include_zone_switch and len(manager.zones) > 1:
        if use_icons:
            switch_badge = create_nav_icon_hint(NAV_ICON_MAP["left_right"], "Switch Panel", badge_style)
            sections.append(
                Div(
                    Span("Panels", cls=combine_classes(font_size.xs, font_weight.semibold, m.r(2))),
                    switch_badge,
                    cls=combine_classes(flex_display, items.center, gap(1), flex_wrap.wrap)
                )
            )
        else:
            prev_key = format_key_for_display(manager.prev_zone_key)
            next_key = format_key_for_display(manager.next_zone_key)
            switch_hints = [(f"{prev_key}/{next_key}", "Switch Panel")]
            sections.append(render_hint_group("Panels", switch_hints, badge_style))
    
    # Action hints (grouped by hint_group)
    action_hints = render_hints_from_actions(manager.actions, badge_style)
    sections.append(action_hints)
    
    return Div(
        *sections,
        id=container_id,
        cls=combine_classes(flex_display, flex_wrap.wrap, gap(4), font_size.sm)
    )

In [ ]:
# Test full keyboard hints with icons (default)
from cjm_fasthtml_keyboard_navigation.core.focus_zone import FocusZone

zone1 = FocusZone(id="z1")
zone2 = FocusZone(id="z2")

manager = ZoneManager(
    zones=(zone1, zone2),
    actions=(
        KeyAction(key=" ", htmx_trigger="toggle", description="Select", hint_group="Selection"),
    )
)

hints = render_keyboard_hints(manager)
html = to_xml(hints)

assert 'id="kb-hints"' in html
assert "Navigate" in html
assert "Switch Panel" in html  # Two zones = show zone switch
assert "Select" in html
assert "svg" in html  # Icons are SVGs

# Test without icons
hints_no_icons = render_keyboard_hints(manager, use_icons=False)
html_no_icons = to_xml(hints_no_icons)
assert "↑/↓" in html_no_icons  # Text arrows when icons disabled

In [ ]:
# Single zone = no zone switch hint
single_manager = ZoneManager(zones=(zone1,), actions=())
hints = render_keyboard_hints(single_manager)
html = to_xml(hints)
assert "Switch Panel" not in html

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()